In [14]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('partition2.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)


# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

cv.imshow('sure_bg',sure_bg)
cv.imshow('sure_fg',sure_fg)

cv.imshow('unknown',unknown)

# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0


markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv.imshow('img',img)
cv.waitKey(0)
cv.destroyAllWindows()


# titles = ['Original Image','thresh']
# plt.figure(dpi=800)

# img = img[:, :, ::-1]

# images = [img, thresh]
# for i in range(2):
#     plt.subplot(1,2,i+1),plt.imshow(images[i],'gray')
#     plt.title(titles[i])
#     plt.xticks([]),plt.yticks([])
# plt.show()